# Convert voxel size of the input images

## Requirements

- A folder with images to convert

<hr style="height:2px;">

## Config

<hr style="height:2px;">

### The following code imports and declares functions used for the processing:

In [1]:
#################################
#  Don't modify the code below  #
#################################

import intake_io
import os
import numpy as np
from tqdm import tqdm
from am_utils.utils import walk_dir

### Please provide data paths:

`input_dir`: directory with images to be segmented

`output_dir`: directory to save the segmentation results; ROI masks will be added as an extra channel to the input image and saved in this directory

In [2]:
input_dir = "../example_data/stacks"
output_dir = "../test_output/converted"

<hr style="height:2px;">

## Setup

<hr style="height:2px;">

### The following code lists all image files in the input directory:

In [3]:
#################################
#  Don't modify the code below  #
#################################

samples = walk_dir(input_dir)

print(f'\n{len(samples)} images were found:')
for i in range(len(samples)):
    print(i, samples[i])


2 images were found:
0 ../example_data/stacks/sample1_position1.tif
1 ../example_data/stacks/sample1_position2.tif


### The following code loads a random image

In [4]:
#################################
#  Don't modify the code below  #
#################################
sample_index = np.random.randint(len(samples))
sample = samples[sample_index]
dataset = intake_io.imload(sample)
print(dataset, '\n')
if 'c' in dataset.dims:
    nchannels = len(dataset['c'].data)
    chnames = dataset['c'].data
else:
    nchannels = 1
    chnames = [0]
print(rf"Number of channels: {nchannels}")

spacing = intake_io.get_spacing(dataset)

print('The following voxel size was detected:')
if 'z' in dataset.dims:
    dims = ['z', 'y', 'x']
else:
    dims = ['y', 'x']
for s, c in zip(spacing, dims):
    print(rf"{c}: {s}")

<xarray.Dataset>
Dimensions:  (y: 326, x: 326, z: 5, c: 3)
Coordinates:
  * y        (y) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * x        (x) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * z        (z) float64 0.0 0.2 0.4 0.6 0.8
Dimensions without coordinates: c
Data variables:
    image    (c, z, y, x) uint16 99 102 99 102 103 99 ... 101 102 96 100 104 103
Attributes:
    metadata:  {'spacing_units': {'z': 'µm', 'y': 'µm', 'x': 'µm'}} 

Number of channels: 3
The following voxel size was detected:
z: 0.2
y: 0.11000000105937943
x: 0.11000000105937943


<hr style="height:0.5px;">

### Please specify the correct voxel size 

Specify the correct voxel size (pixel size and z-stepping) in microns if the above values are wrong.

Keep `None`, if the value loaded from the metadata is correct.

In [5]:
z = 0.25
y = None
x = None

### The following code loads a random stack with correct voxel size

In [6]:
#################################
#  Don't modify the code below  #
#################################

meta = intake_io.imload(sample, metadata_only=True)['metadata']
for k, v in zip(['x', 'y', 'z'], [x, y, z]):
    if k in meta['spacing'].keys() and v is not None:
        meta['spacing'][k] = v

if 'z' in dataset.dims:
    spacing = [z, y, x]
else:
    spacing = [y, x]
dataset = intake_io.imload(sample, metadata=meta)
print(dataset)

spacing = intake_io.get_spacing(dataset)

print('\nThe new voxel size:')
if 'z' in dataset.dims:
    dims = ['z', 'y', 'x']
else:
    dims = ['y', 'x']
for s, c in zip(spacing, dims):
    print(rf"{c}: {s}")

<xarray.Dataset>
Dimensions:  (y: 326, x: 326, z: 5, c: 3)
Coordinates:
  * y        (y) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * x        (x) float64 0.0 0.11 0.22 0.33 0.44 ... 35.42 35.53 35.64 35.75
  * z        (z) float64 0.0 0.25 0.5 0.75 1.0
Dimensions without coordinates: c
Data variables:
    image    (c, z, y, x) uint16 99 102 99 102 103 99 ... 101 102 96 100 104 103
Attributes:
    metadata:  {'spacing_units': {'y': 'µm', 'x': 'µm', 'z': 'µm'}}

The new voxel size:
z: 0.25
y: 0.11000000105937943
x: 0.11000000105937943


<hr style="height:2px;">

## Processing

<hr style="height:2px;">

### The following code converts all images in the input folder

In [7]:
for sample in tqdm(samples):
    fn_out = sample.replace(input_dir.rstrip('/'), output_dir.rstrip('/'))
    dataset = intake_io.imload(sample, metadata=meta)
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    try:
        intake_io.imsave(dataset, fn_out)
    except UnicodeDecodeError:
        dataset['image'].metadata['spacing_units'] = {}
        intake_io.imsave(dataset, fn_out)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 109.93it/s]
